# Init SparkContext

In [1]:
import os
from datetime import datetime
from pyspark.sql import SparkSession, SQLContext
import pyspark.sql.functions as F
from pyspark.sql.types import FloatType
from pyspark.storagelevel import StorageLevel

In [2]:
spark = (SparkSession.builder.appName("spark-101-{}".format(datetime.today()))
        .master("spark://spark-master:7077")      
        .getOrCreate())

sqlContext = SQLContext(spark)

/usr/local/spark/python/pyspark/sql/context.py:112: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [3]:
sc = spark.sparkContext
sc

<SparkContext master=spark://spark-master:7077 appName=spark-101-2023-09-08 03:33:39.507096>

# Reading CSV file

In [5]:
base_path = "s3a://warehouse/tpch_data"

In [ ]:
%%time
filename = "h_lineitem.dsv"

df_lineitem = (
    spark.read.option("delimiter", "|")
    .option("header", True)
    .option("inferSchema" , True)
    .csv(os.path.join(base_path, filename))
)

In [6]:
df_lineitem.printSchema()

root
 |-- L_ORDERKEY: integer (nullable = true)
 |-- L_PARTKEY: integer (nullable = true)
 |-- L_SUPPKEY: integer (nullable = true)
 |-- L_LINENUMBER: integer (nullable = true)
 |-- L_QUANTITY: integer (nullable = true)
 |-- L_EXTENDEDPRICE: string (nullable = true)
 |-- L_DISCOUNT: string (nullable = true)
 |-- L_TAX: string (nullable = true)
 |-- L_RETURNFLAG: string (nullable = true)
 |-- L_LINESTATUS: string (nullable = true)
 |-- L_SHIPDATE: string (nullable = true)
 |-- L_COMMITDATE: string (nullable = true)
 |-- L_RECEIPTDATE: string (nullable = true)
 |-- L_SHIPINSTRUCT: string (nullable = true)
 |-- L_SHIPMODE: string (nullable = true)
 |-- L_COMMENT: string (nullable = true)



# Writing parquet files

In [7]:
df_lineitem.rdd.getNumPartitions()

14

In [8]:
table_name = "h_lineitem"
output_path = os.path.join(base_path, "parquet", table_name)

In [9]:
%%time
df_lineitem.write.parquet(output_path, mode="overwrite")

CPU times: user 9.3 ms, sys: 3.11 ms, total: 12.4 ms
Wall time: 40.3 s


In [10]:
df_lineitem.repartition(5).write.parquet(output_path, mode="overwrite")

In [11]:
df_lineitem_compressed = spark.read.parquet(output_path)
df_lineitem_compressed.rdd.getNumPartitions()

5

# Writing JSON files

In [12]:
df_semi_structure = (
    df_lineitem_compressed.groupby("L_ORDERKEY")
    .agg(
        F.collect_list("L_LINENUMBER").alias("items"),
        F.count("L_LINENUMBER").alias("num_items"),
    )
    .orderBy(F.col("num_items").desc())    
)

df_semi_structure.show()

+----------+--------------------+---------+
|L_ORDERKEY|               items|num_items|
+----------+--------------------+---------+
|   5999976|[4, 1, 7, 8, 6, 5...|        8|
|     10180|[3, 5, 1, 6, 4, 2...|        7|
|     10183|[7, 6, 5, 1, 3, 4...|        7|
|     10342|[4, 1, 6, 2, 3, 7...|        7|
|     10211|[5, 1, 4, 7, 3, 2...|        7|
|     10184|[3, 5, 4, 7, 2, 1...|        7|
|     10209|[2, 1, 7, 5, 4, 6...|        7|
|     10210|[3, 5, 2, 6, 7, 1...|        7|
|     10306|[2, 5, 7, 6, 3, 4...|        7|
|     10215|[1, 5, 3, 7, 2, 6...|        7|
|     10216|[5, 2, 4, 7, 6, 1...|        7|
|     10241|[3, 7, 1, 2, 5, 4...|        7|
|     10243|[1, 4, 7, 5, 3, 2...|        7|
|     10247|[5, 1, 4, 6, 3, 2...|        7|
|     10248|[2, 7, 4, 1, 3, 6...|        7|
|     10276|[3, 4, 6, 5, 1, 2...|        7|
|     10309|[2, 3, 6, 5, 7, 4...|        7|
|     10310|[7, 1, 2, 4, 3, 6...|        7|
|     10311|[6, 5, 2, 3, 1, 4...|        7|
|     10338|[2, 5, 6, 1, 4, 3...

In [13]:
df_semi_structure.printSchema()

root
 |-- L_ORDERKEY: integer (nullable = true)
 |-- items: array (nullable = false)
 |    |-- element: integer (containsNull = false)
 |-- num_items: long (nullable = false)



In [14]:
%%time
output_json = os.path.join(base_path, "json", table_name)
df_semi_structure.write.json(output_json, mode="overwrite")

CPU times: user 4.51 ms, sys: 1.5 ms, total: 6.01 ms
Wall time: 14.7 s


# Compare uncompressed/compressed files

In [15]:
%%time
df_lineitem.count()

CPU times: user 1.52 ms, sys: 3.32 ms, total: 4.84 ms
Wall time: 11.8 s


11996782

In [16]:
%%time
df_lineitem_compressed.count()

CPU times: user 0 ns, sys: 2.61 ms, total: 2.61 ms
Wall time: 507 ms


11996782

# Cached vs non-cached

In [18]:
%%time
df_lineitem.cache()
df_lineitem.count()

CPU times: user 9.15 ms, sys: 3.07 ms, total: 12.2 ms
Wall time: 46.5 s


11996782

In [19]:
%%time
df_lineitem.count()

CPU times: user 1.51 ms, sys: 514 µs, total: 2.02 ms
Wall time: 250 ms


11996782

In [20]:
%%time
df_lineitem.unpersist()
df_lineitem.count()

CPU times: user 4.52 ms, sys: 1.53 ms, total: 6.05 ms
Wall time: 12.9 s


11996782

In [21]:
%%time
df_lineitem.persist(StorageLevel.MEMORY_AND_DISK_2)
df_lineitem.count()

CPU times: user 6.17 ms, sys: 9.95 ms, total: 16.1 ms
Wall time: 51.7 s


11996782

In [23]:
%%time
df_lineitem.unpersist()
df_lineitem.persist(StorageLevel.DISK_ONLY)
df_lineitem.count()

CPU times: user 6.98 ms, sys: 6.01 ms, total: 13 ms
Wall time: 47.7 s


11996782

In [24]:
%%time
df_lineitem.count()

CPU times: user 1.56 ms, sys: 6 µs, total: 1.57 ms
Wall time: 518 ms


11996782

# Hive Metastore

In [25]:
df_lineitem_compressed.limit(5).toPandas()

,L_ORDERKEY,L_PARTKEY,L_SUPPKEY,L_LINENUMBER,L_QUANTITY,L_EXTENDEDPRICE,L_DISCOUNT,L_TAX,L_RETURNFLAG,L_LINESTATUS,L_SHIPDATE,L_COMMITDATE,L_RECEIPTDATE,L_SHIPINSTRUCT,L_SHIPMODE,L_COMMENT
0,8582561,376604,11659,1,43,"72265,37","0,07","0,04",R,F,15.04.92,28.03.92,25.04.92,COLLECT COD,AIR,carefully unusual foxes was
1,9332744,198681,3691,1,48,"85424,64","0,09","0,07",R,F,06.04.92,10.04.92,12.04.92,TAKE BACK RETURN,MAIL,quickly even excuses impres
2,6162691,102019,2020,5,24,"24504,24","0,04","0,03",R,F,25.02.92,12.03.92,29.02.92,COLLECT COD,RAIL,thinly ironic forges doze p
3,8564931,370250,5305,1,43,"56770,32","0,07","0,04",A,F,19.03.92,02.03.92,29.03.92,COLLECT COD,AIR,carefully even foxes was qu
4,5014790,81292,11301,6,23,"29285,67","0,09","0,03",A,F,18.01.92,08.03.92,20.01.92,NONE,AIR,carefully regular ideas hag


In [26]:
# register temporary view
table_name = "h_lineitem"
df_lineitem_compressed.createOrReplaceTempView(table_name)

In [27]:
spark.sql("SHOW TABLES").show()

+---------+----------+-----------+
|namespace| tableName|isTemporary|
+---------+----------+-----------+
|         |h_lineitem|      false|
+---------+----------+-----------+



In [28]:
spark.sql("SELECT * FROM h_lineitem LIMIT 5").show()

+----------+---------+---------+------------+----------+---------------+----------+-----+------------+------------+----------+------------+-------------+--------------------+----------+--------------------+
|L_ORDERKEY|L_PARTKEY|L_SUPPKEY|L_LINENUMBER|L_QUANTITY|L_EXTENDEDPRICE|L_DISCOUNT|L_TAX|L_RETURNFLAG|L_LINESTATUS|L_SHIPDATE|L_COMMITDATE|L_RECEIPTDATE|      L_SHIPINSTRUCT|L_SHIPMODE|           L_COMMENT|
+----------+---------+---------+------------+----------+---------------+----------+-----+------------+------------+----------+------------+-------------+--------------------+----------+--------------------+
|   8582561|   376604|    11659|           1|        43|       72265,37|      0,07| 0,04|           R|           F|  15.04.92|    28.03.92|     25.04.92|COLLECT COD      ...|AIR       |carefully unusual...|
|   9332744|   198681|     3691|           1|        48|       85424,64|      0,09| 0,07|           R|           F|  06.04.92|    10.04.92|     12.04.92|TAKE BACK RETURN ..

# Dataframe transformation

## SELECT

In [29]:
df_lineitem_compressed.select("L_ORDERKEY", "L_LINENUMBER", "L_QUANTITY").show(5)

+----------+------------+----------+
|L_ORDERKEY|L_LINENUMBER|L_QUANTITY|
+----------+------------+----------+
|   8582561|           1|        43|
|   9332744|           1|        48|
|   6162691|           5|        24|
|   8564931|           1|        43|
|   5014790|           6|        23|
+----------+------------+----------+
only showing top 5 rows



In [30]:
ls_selected_cols = ["L_ORDERKEY", "L_LINENUMBER", "L_QUANTITY"]
df_lineitem_compressed.select(ls_selected_cols).show(5)

+----------+------------+----------+
|L_ORDERKEY|L_LINENUMBER|L_QUANTITY|
+----------+------------+----------+
|   8582561|           1|        43|
|   9332744|           1|        48|
|   6162691|           5|        24|
|   8564931|           1|        43|
|   5014790|           6|        23|
+----------+------------+----------+
only showing top 5 rows



In [31]:
df_lineitem_compressed.selectExpr("L_ORDERKEY AS order_no", "L_LINENUMBER AS line_no", "L_QUANTITY AS quantity").show(5)

+--------+-------+--------+
|order_no|line_no|quantity|
+--------+-------+--------+
| 8582561|      1|      43|
| 9332744|      1|      48|
| 6162691|      5|      24|
| 8564931|      1|      43|
| 5014790|      6|      23|
+--------+-------+--------+
only showing top 5 rows



## WHERE

In [32]:
df_lineitem_compressed.filter("L_ORDERKEY = 1090753").show()

+----------+---------+---------+------------+----------+---------------+----------+-----+------------+------------+----------+------------+-------------+--------------------+----------+--------------------+
|L_ORDERKEY|L_PARTKEY|L_SUPPKEY|L_LINENUMBER|L_QUANTITY|L_EXTENDEDPRICE|L_DISCOUNT|L_TAX|L_RETURNFLAG|L_LINESTATUS|L_SHIPDATE|L_COMMITDATE|L_RECEIPTDATE|      L_SHIPINSTRUCT|L_SHIPMODE|           L_COMMENT|
+----------+---------+---------+------------+----------+---------------+----------+-----+------------+------------+----------+------------+-------------+--------------------+----------+--------------------+
|   1090753|   291416|     1445|           2|        41|        57703,4|      0,08| 0,07|           A|           F|  03.01.93|    20.12.92|     24.01.93|NONE             ...|FOB       |bold requests try...|
|   1090753|    91558|     6571|           6|        14|        21693,7|       0,1| 0,05|           R|           F|  16.12.92|    22.11.92|     12.01.93|DELIVER IN PERSON..

In [33]:
df_lineitem_compressed.where("L_ORDERKEY = 1090753 AND L_LINENUMBER < 4").show()

+----------+---------+---------+------------+----------+---------------+----------+-----+------------+------------+----------+------------+-------------+--------------------+----------+--------------------+
|L_ORDERKEY|L_PARTKEY|L_SUPPKEY|L_LINENUMBER|L_QUANTITY|L_EXTENDEDPRICE|L_DISCOUNT|L_TAX|L_RETURNFLAG|L_LINESTATUS|L_SHIPDATE|L_COMMITDATE|L_RECEIPTDATE|      L_SHIPINSTRUCT|L_SHIPMODE|           L_COMMENT|
+----------+---------+---------+------------+----------+---------------+----------+-----+------------+------------+----------+------------+-------------+--------------------+----------+--------------------+
|   1090753|   291416|     1445|           2|        41|        57703,4|      0,08| 0,07|           A|           F|  03.01.93|    20.12.92|     24.01.93|NONE             ...|FOB       |bold requests try...|
|   1090753|   215598|      609|           3|        14|       21190,12|      0,01|    0|           R|           F|  15.11.92|    19.11.92|     17.11.92|NONE             ..

In [34]:
df_lineitem_compressed.where((F.col("L_ORDERKEY") == 1090753) & (F.col("L_LINENUMBER") < F.lit(4))).show()

+----------+---------+---------+------------+----------+---------------+----------+-----+------------+------------+----------+------------+-------------+--------------------+----------+--------------------+
|L_ORDERKEY|L_PARTKEY|L_SUPPKEY|L_LINENUMBER|L_QUANTITY|L_EXTENDEDPRICE|L_DISCOUNT|L_TAX|L_RETURNFLAG|L_LINESTATUS|L_SHIPDATE|L_COMMITDATE|L_RECEIPTDATE|      L_SHIPINSTRUCT|L_SHIPMODE|           L_COMMENT|
+----------+---------+---------+------------+----------+---------------+----------+-----+------------+------------+----------+------------+-------------+--------------------+----------+--------------------+
|   1090753|   291416|     1445|           2|        41|        57703,4|      0,08| 0,07|           A|           F|  03.01.93|    20.12.92|     24.01.93|NONE             ...|FOB       |bold requests try...|
|   1090753|   215598|      609|           3|        14|       21190,12|      0,01|    0|           R|           F|  15.11.92|    19.11.92|     17.11.92|NONE             ..

## ORDER BY

In [35]:
df_filtered = df_lineitem_compressed.where("L_ORDERKEY = 1090753 AND L_LINENUMBER < 4")
df_filtered.orderBy("L_LINENUMBER").show()

+----------+---------+---------+------------+----------+---------------+----------+-----+------------+------------+----------+------------+-------------+--------------------+----------+--------------------+
|L_ORDERKEY|L_PARTKEY|L_SUPPKEY|L_LINENUMBER|L_QUANTITY|L_EXTENDEDPRICE|L_DISCOUNT|L_TAX|L_RETURNFLAG|L_LINESTATUS|L_SHIPDATE|L_COMMITDATE|L_RECEIPTDATE|      L_SHIPINSTRUCT|L_SHIPMODE|           L_COMMENT|
+----------+---------+---------+------------+----------+---------------+----------+-----+------------+------------+----------+------------+-------------+--------------------+----------+--------------------+
|   1090753|   226447|    11459|           1|        45|       61804,35|      0,07| 0,04|           A|           F|  25.10.92|    02.01.93|     15.11.92|DELIVER IN PERSON...|REG AIR   |fluffily final co...|
|   1090753|   291416|     1445|           2|        41|        57703,4|      0,08| 0,07|           A|           F|  03.01.93|    20.12.92|     24.01.93|NONE             ..

In [36]:
df_filtered.orderBy("L_LINENUMBER", ascending=False).show()

+----------+---------+---------+------------+----------+---------------+----------+-----+------------+------------+----------+------------+-------------+--------------------+----------+--------------------+
|L_ORDERKEY|L_PARTKEY|L_SUPPKEY|L_LINENUMBER|L_QUANTITY|L_EXTENDEDPRICE|L_DISCOUNT|L_TAX|L_RETURNFLAG|L_LINESTATUS|L_SHIPDATE|L_COMMITDATE|L_RECEIPTDATE|      L_SHIPINSTRUCT|L_SHIPMODE|           L_COMMENT|
+----------+---------+---------+------------+----------+---------------+----------+-----+------------+------------+----------+------------+-------------+--------------------+----------+--------------------+
|   1090753|   215598|      609|           3|        14|       21190,12|      0,01|    0|           R|           F|  15.11.92|    19.11.92|     17.11.92|NONE             ...|FOB       |fluffily silent f...|
|   1090753|   291416|     1445|           2|        41|        57703,4|      0,08| 0,07|           A|           F|  03.01.93|    20.12.92|     24.01.93|NONE             ..

## New columns

In [37]:
df_filtered = df_lineitem_compressed.where("L_ORDERKEY = 1090753")
df_filtered = df_filtered.selectExpr("L_ORDERKEY AS order_no", "L_LINENUMBER AS line_no", "L_EXTENDEDPRICE", "CAST(L_QUANTITY AS INT) AS quantity")
df_filtered.show()

+--------+-------+---------------+--------+
|order_no|line_no|L_EXTENDEDPRICE|quantity|
+--------+-------+---------------+--------+
| 1090753|      2|        57703,4|      41|
| 1090753|      6|        21693,7|      14|
| 1090753|      3|       21190,12|      14|
| 1090753|      5|         4344,2|       4|
| 1090753|      1|       61804,35|      45|
| 1090753|      4|        30188,6|      26|
+--------+-------+---------------+--------+



In [38]:
df_filtered = df_filtered.withColumn("amount", F.regexp_replace("L_EXTENDEDPRICE", ",", ".").cast(FloatType()))
df_filtered.show()

+--------+-------+---------------+--------+--------+
|order_no|line_no|L_EXTENDEDPRICE|quantity|  amount|
+--------+-------+---------------+--------+--------+
| 1090753|      2|        57703,4|      41| 57703.4|
| 1090753|      6|        21693,7|      14| 21693.7|
| 1090753|      3|       21190,12|      14|21190.12|
| 1090753|      5|         4344,2|       4|  4344.2|
| 1090753|      1|       61804,35|      45|61804.35|
| 1090753|      4|        30188,6|      26| 30188.6|
+--------+-------+---------------+--------+--------+



In [39]:
df_filtered = df_filtered.withColumn("total_price", F.col("amount") * F.col("quantity"))
df_filtered.show()

+--------+-------+---------------+--------+--------+-----------+
|order_no|line_no|L_EXTENDEDPRICE|quantity|  amount|total_price|
+--------+-------+---------------+--------+--------+-----------+
| 1090753|      2|        57703,4|      41| 57703.4|  2365839.2|
| 1090753|      6|        21693,7|      14| 21693.7|  303711.78|
| 1090753|      3|       21190,12|      14|21190.12|  296661.66|
| 1090753|      5|         4344,2|       4|  4344.2|    17376.8|
| 1090753|      1|       61804,35|      45|61804.35|  2781195.8|
| 1090753|      4|        30188,6|      26| 30188.6|  784903.56|
+--------+-------+---------------+--------+--------+-----------+



## GROUP BY, AGG: sum(), min(), max(), avg()

In [40]:
df_agg = (
    df_filtered.groupBy("order_no")
    .agg(
        F.min("total_price").alias("min_price"),
        F.max("total_price").alias("max_price"),
        F.avg("total_price").alias("avg_price"),
        F.sum("total_price").alias("total_price"),
        F.count("line_no").alias("num_items")
    )
)

df_agg = df_agg.selectExpr(["*"] + ["(total_price / num_items) AS values_per_items"])
df_agg.show()

+--------+---------+---------+------------------+----------------+---------+------------------+
|order_no|min_price|max_price|         avg_price|     total_price|num_items|  values_per_items|
+--------+---------+---------+------------------+----------------+---------+------------------+
| 1090753|  17376.8|2781195.8|1091614.8001302083|6549688.80078125|        6|1091614.8001302083|
+--------+---------+---------+------------------+----------------+---------+------------------+

